## Dependencies

In [1]:
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

## VGG19 (pre-trained)

In [2]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

80134624/80134624 [==============================] - 1s 0us/step


## Freeze all

In [3]:
for layer in vgg19.layers:
    layer.trainable = False

## Add custom classification layers

In [4]:
model = Sequential()
model.add(vgg19)

model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(2, activation='softmax', name='predictions'))

## Compile

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#### Prepare the data for training

DATASET [here](https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog/home)

In [17]:
from google.colab import files #import the files fron colab

In [18]:
files.upload() #upload your kaggle.json file form your PC

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"markapiezon","key":"1bba70ee37faa803149601f9acbd1c24"}'}

In [19]:
! mkdir ~/.kaggle #creating folder

In [20]:
! cp kaggle.json ~/.kaggle/ #copying kaggle.json

In [21]:
! chmod 600 ~/.kaggle/kaggle.json #reading the file with full access

In [22]:
! kaggle datasets download -d dansbecker/hot-dog-not-hot-dog

 91% 81.0M/89.3M [00:00<00:00, 79.7MB/s]
100% 89.3M/89.3M [00:00<00:00, 94.4MB/s]


In [25]:
import zipfile
import pandas as pd 

# train = pd.read_csv("/content/train.csv.zip" ,compression='zip') #reading the train.csv zip file and store in train variable
# test = pd.read_csv("/content/test.csv.zip" ,compression='zip') #reading the test.csv zip flie and store in test variable

In [ ]:
data_r = zipfile.ZipFile('./hot-dog-not-hot-dog.zip', 'r')
data_r.printdir()

In [28]:
data_r.extractall()

In [30]:
import os

In [31]:
os.getcwd()

'/content'

In [7]:
# Instantiate two image generator classes:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(rescale=1.0/255,
                                   data_format='channels_last')

In [8]:
batch_size=32

In [32]:
train_generator = train_datagen.flow_from_directory(
    directory='./train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory='./test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [33]:
model.fit_generator(train_generator, steps_per_epoch=15, 
                    epochs=16,
                    validation_data=valid_generator,
                    validation_steps=15)

<ipython-input-33-c93ef18e0c71>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=15,


Epoch 1/16
 7/15 [=============>................] - ETA: 2:33 - loss: 0.7919 - accuracy: 0.5938

KeyboardInterrupt: ignored